In [54]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce


pd.set_option("max_rows", None)
pd.set_option("max_columns", None)
import warnings
warnings.filterwarnings('ignore')

In [55]:
energy_usage_selected_cities = pd.read_csv('Energy_Usage_2010_preprocessed.csv')

In [56]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import category_encoders as ce   # version 1.2.8
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin

class StandardScalarMultiple(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        super().__init__()
        self.columns = columns
        self.normalizer = None

    def fit(self, X, y=None):
        self.normalizer = StandardScaler()
        self.normalizer.fit(X[self.columns])

        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()

        normalized_vals = self.normalizer.transform(X[self.columns])
        
        for i in range(0, len(self.columns)):
            col = [row[i] for row in normalized_vals]
            X_copy[self.columns[i]] = col
        

        return X_copy

### Modelling

Features used for the model
- Building Type
- Building Sub Type
- Total population
- Average Stories
- Average Building Age
- Average Housesize
- Electricity Accounts

Pipeline
- One Hot Encoding
- Standardization

Model used
- Gradient Boosting



In [57]:
from sklearn.model_selection import train_test_split



one_hot_encoder = ce.OneHotEncoder(cols=[ 'COMMUNITY AREA NAME', 'BUILDING TYPE', 'BUILDING_SUBTYPE'])
standard_scaler = StandardScalarMultiple(columns=['TOTAL POPULATION', 'AVERAGE STORIES', 
                                                  'AVERAGE BUILDING AGE', 'AVERAGE HOUSESIZE', 'ELECTRICITY ACCOUNTS'
                                                  ])

energy_usage_select_cities_selected_cols = \
    energy_usage_selected_cities[['BUILDING TYPE', 'BUILDING_SUBTYPE', 'COMMUNITY AREA NAME',
                                    'TOTAL POPULATION', 'AVERAGE STORIES', 
                                                  'AVERAGE BUILDING AGE', 'AVERAGE HOUSESIZE', 'RENTER-OCCUPIED HOUSING PERCENTAGE',
                                    'ELECTRICITY ACCOUNTS', 'KWH MEAN 2010'
                                                  ]]
# energy_usage_select_cities_nonull_selected_cols=energy_usage_select_cities_nonull_selected_cols[energy_usage_select_cities_nonull_selected_cols['BUILDING TYPE']!='Industrial']

X = energy_usage_select_cities_selected_cols[['COMMUNITY AREA NAME', 'BUILDING TYPE', 'BUILDING_SUBTYPE',
                                    'TOTAL POPULATION', 'AVERAGE STORIES', 
                                                  'AVERAGE BUILDING AGE', 'AVERAGE HOUSESIZE', 'RENTER-OCCUPIED HOUSING PERCENTAGE', 'ELECTRICITY ACCOUNTS']]
y = energy_usage_select_cities_selected_cols['KWH MEAN 2010']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=611, stratify=X['COMMUNITY AREA NAME'])


### Apply Pipeline and perform Gradient Boosting

- Applied 4 fold cross validation

### Evaluation using percentile rank

Compute percentile rank of true and predicted values
- Compute rmse score on 4 quantiles and check whether model is predicting very low and very high energy consumption
- rmse percentile rank is consistent across all the folds which implies the model is not overfitting

In [58]:
from sklearn.model_selection import train_test_split

def rmse_percentile_rank(y_test, pred):
    
    evaluation_df = pd.DataFrame({
        'true': y_test,
        'pred': pred,
        'true_rank': pd.Series(y_test).rank(pct=True),
        'pred_rank': pd.Series(pred).rank(pct=True)
    })
    evaluation_df['quantile'] = pd.qcut(evaluation_df.true, 4, labels=False)
    evaluation_df['diff_percentile'] = np.square(evaluation_df['true_rank']-evaluation_df['pred_rank'])
    
    return np.sqrt(mean_squared_error(evaluation_df['true_rank'], evaluation_df['pred_rank']))


pipe = Pipeline([
                 ('one_hot_encoder', one_hot_encoder),
                 ('standard_scaler', standard_scaler)
                ])

pipe_fit = pipe.fit(X_train)

X_train_transformed = pipe_fit.transform(X_train)

X_train_transformed = np.array(X_train_transformed.to_numpy())

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4)
skf.get_n_splits(X, y)

y_train = np.array(y_train)

count = 0
for train_index, test_index in skf.split(X_train_transformed, X_train['BUILDING TYPE']):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_cross_train, X_cross_test = X_train_transformed[train_index], X_train_transformed[test_index]
    y_cross_train, y_cross_test = y_train[train_index], y_train[test_index]
    
    lm = GradientBoostingRegressor().fit(X_cross_train, y_cross_train)
    pred = lm.predict(X_cross_test)
    
    count+=1
    print(rmse_percentile_rank(y_cross_test, pred), ' --- Percentile rmse rank of Fold split ', count)
    

0.3024555653656401  --- Percentile rmse rank of Fold split  1
0.3032854185560264  --- Percentile rmse rank of Fold split  2
0.3049009562896139  --- Percentile rmse rank of Fold split  3
0.29505755179191645  --- Percentile rmse rank of Fold split  4


In [59]:
mdl = GradientBoostingRegressor().fit(X_train_transformed, y_train)

In [60]:
X_test_transformed = pipe_fit.transform(X_test)
pred = mdl.predict(X_test_transformed)

### Evaluation using percentile rank

Compute percentile rank of true and predicted values
- Compute rmse score on 4 quantiles and check whether model is predicting very low and very high energy consumption
- rmse of overall percentile rank of predicted and true energy consumption is low, implying model predicts energy consumption accurately.
- Rmse of predicted and true percentile rank is low, which implies that model can predict energy consumption percentiles accurately

In [61]:
evaluation_df = pd.DataFrame({
    'true': y_test.values,
    'pred': pred,
    'true_rank': y_test.rank(pct=True).values,
    'pred_rank': pd.Series(pred).rank(pct=True).values
})
evaluation_df['quantile'] = pd.qcut(evaluation_df.true, 4, labels=False)
evaluation_df['diff_percentile'] = np.square(evaluation_df['true_rank']-evaluation_df['pred_rank'])

print(np.sqrt(mean_squared_error(evaluation_df['true_rank'], evaluation_df['pred_rank'])))

0.2895792908553612


### RMSE based on quantiles
- Rmse of predicted and true percentile rank is low, which implies that model can predict energy consumption percentiles accurately

In [62]:
np.sqrt(evaluation_df.groupby('quantile').agg({'diff_percentile':['mean']}))

,diff_percentile
,mean
quantile,
0,0.381271
1,0.248154
2,0.276186
3,0.228401


### Evaluation using percentile rank for West Town Area

Compute percentile rank of true and predicted values
- Compute rmse score on 4 quantiles and check whether model is predicting very low and very high energy consumption
- rmse of overall percentile rank of predicted and true energy consumption is low, implying model predicts energy consumption accurately.
- Rmse of predicted and true percentile rank is low, which implies that model can predict energy consumption percentiles accurately

In [63]:
X_test['predicted_energy_usage'] = pred
X_test['actual_energy_usage'] = y_test

In [64]:
energy_usage_west_town = X_test[X_test['COMMUNITY AREA NAME']=='West Town']

In [65]:
energy_usage_west_town['predicted_energy_usage_rank'] = energy_usage_west_town['predicted_energy_usage'].rank(pct=True)
energy_usage_west_town['actual_energy_usage_rank'] = energy_usage_west_town['actual_energy_usage'].rank(pct=True)
energy_usage_west_town['quantile'] = pd.qcut(energy_usage_west_town.predicted_energy_usage, 4, labels=False)
energy_usage_west_town['diff_percentile'] = np.square(energy_usage_west_town['actual_energy_usage_rank']-energy_usage_west_town['predicted_energy_usage_rank'])


In [66]:
print(np.sqrt(mean_squared_error(energy_usage_west_town['actual_energy_usage_rank'], energy_usage_west_town['predicted_energy_usage_rank'])))

0.2966704148248639


In [67]:
np.sqrt(evaluation_df.groupby('quantile').agg({'diff_percentile':['mean']}))

,diff_percentile
,mean
quantile,
0,0.381271
1,0.248154
2,0.276186
3,0.228401
